In [1]:
import os
import numpy as np
import pandas as pd

from src import RESULT_DIR
from src.utils import MCC, DIS

In [2]:
classifier = 'RandomForest' # RandomForest, LightGBM
alignment = 'temporal' # geometric, temporal, warp

In [3]:
if classifier == 'LightGBM': 
        cols = ['fold_num', 'trial', 'split_num', 'num_leaves', 
                'min_child_samples', 'threshold', 'num_trees', 
                'tn', 'fp', 'fn', 'tp','time']
        cc = ['fold_num', 'trial', 'num_leaves', 'min_child_samples', 
              'threshold']
        
elif classifier == 'RandomForest': 
        cols = ['num_fold', 'step', 'num_split', 'trees', 
                'max_depth', 'threshold',  
                'tn', 'fp', 'fn', 'tp','time']
        cc = ['num_fold', 'step', 'trees', 'max_depth', 
              'threshold']

In [5]:
res_dir = os.path.join(RESULT_DIR, classifier, alignment, 'classification')
res_df = {}
res_mcc = []
for fold in range(1,6):
    csv_dir = os.path.join(res_dir, 'results_fold{0:02d}.csv'.format(fold))
    dd = pd.read_csv(csv_dir, index_col=0)
    dd = dd[cols]
    dd.reset_index(drop=True, inplace=True)
    #dd['trial'] = np.repeat(list(range(int(dd.shape[0] / 8))),8).astype(int)
    dd = dd.groupby(cc).aggregate({
                    'tn':np.sum,'fp':np.sum,'fn':np.sum,
                    'tp':np.sum,'time':np.average}).reset_index()
    dd['MCC'] = dd.apply(lambda x: MCC(x['tn'],x['fp'],x['fn'],x['tp']), axis=1)
    dd_max = dd.loc[dd['MCC'] == dd['MCC'].max(), :]
    #res_df[fold] = {'num_leaves':dd_max['num_leaves'].values[0], 'min_child_samples':dd_max['min_child_samples'].values[0], 'threshold':dd_max['threshold'].values[0]}
    print('Fold {0}:  {1:>3} iterations - max it. = {6}, max.MCC = {2:.4f}  [{3}  {4}  {5:0.4f}]'.format(fold,dd[cc[1]].max()+1,dd['MCC'].max(),
                                                                                                         dd_max[cc[2]].values[0],
                                                                                                         dd_max[cc[3]].values[0],
                                                                                                         dd_max['threshold'].values[0],
                                                                                                         dd_max[cc[1]].values[0]))
    #res_mcc.append(dd['MCC'].max())


Fold 1:  300 iterations - max it. = 120, max.MCC = 0.7272  [200  46  38.0000]
Fold 2:  272 iterations - max it. = 143, max.MCC = 0.7370  [200  44  38.0000]
Fold 3:   26 iterations - max it. = 18, max.MCC = 0.7259  [200  84  37.0000]
Fold 4:   97 iterations - max it. = 81, max.MCC = 0.7517  [180  51  40.0000]
Fold 5:   28 iterations - max it. = 20, max.MCC = 0.7196  [200  165  40.0000]


In [ ]:
Fold 1:  300 iterations - max it. = 120, max.MCC = 0.7272  [200  46  38.0000]
Fold 2:  272 iterations - max it. = 143, max.MCC = 0.7370  [200  44  38.0000]
Fold 3:   26 iterations - max it. = 18, max.MCC = 0.7259  [200  84  37.0000]
Fold 4:   71 iterations - max it. = 50, max.MCC = 0.7513  [200  148  39.0000]
Fold 5:   22 iterations - max it. = 20, max.MCC = 0.7196  [200  165  40.0000]

In [11]:
res_dir = os.path.join(RESULT_DIR, classifier, alignment, 'classification')
res_df = {}
res_mcc = []
for fold in range(1,10):
    csv_dir = os.path.join(res_dir, 'results_fold{0:02d}.csv'.format(fold))
    dd = pd.read_csv(csv_dir, index_col=0)
    dd = dd[cols]
    dd.reset_index(drop=True, inplace=True)
    #dd['trial'] = np.repeat(list(range(int(dd.shape[0] / 8))),8).astype(int)
    dd = dd.groupby(cc).aggregate({
                    'tn':np.sum,'fp':np.sum,'fn':np.sum,
                    'tp':np.sum,'time':np.average}).reset_index()
    dd['MCC'] = dd.apply(lambda x: MCC(x['tn'],x['fp'],x['fn'],x['tp']), axis=1)
    dd['DIS'] = dd.apply(lambda x: DIS(x['tn'],x['fp'],x['fn'],x['tp']), axis=1)
    dd_max = dd.loc[dd['MCC'] == dd['MCC'].max(), :]
    #res_df[fold] = {'num_leaves':dd_max['num_leaves'].values[0], 'min_child_samples':dd_max['min_child_samples'].values[0], 'threshold':dd_max['threshold'].values[0]}
    print('\\textbf{{{0}}} & {1:>3} & {2:>3} & {3:>3} & {4:0.2f} & {5:0.4f} & {6:0.4f} \\\\'.format(fold,
                                                                                            dd[cc[1]].max()+1,
                                                                                            dd_max[cc[2]].values[0],
                                                                                            dd_max[cc[3]].values[0],
                                                                                            dd_max['threshold'].values[0]/100,
                                                                                            dd_max['MCC'].values[0],
                                                                                            dd_max['DIS'].values[0],
                                                                                        ))

\textbf{1} & 288 & 200 &  50 & 0.39 & 0.7263 & 0.2003 \\
\textbf{2} & 300 & 200 &  44 & 0.39 & 0.7331 & 0.1940 \\
\textbf{3} & 300 & 200 &  68 & 0.38 & 0.7244 & 0.2001 \\
\textbf{4} & 300 & 200 &  50 & 0.40 & 0.7527 & 0.1856 \\
\textbf{5} & 284 & 200 &  78 & 0.41 & 0.7178 & 0.2151 \\
\textbf{6} & 300 & 200 &  73 & 0.40 & 0.7459 & 0.1830 \\
\textbf{7} & 300 & 200 &  57 & 0.40 & 0.7739 & 0.1640 \\
\textbf{8} & 287 & 200 &  52 & 0.38 & 0.7380 & 0.1908 \\
\textbf{9} & 300 & 200 &  64 & 0.39 & 0.7330 & 0.1991 \\


In [20]:
dd_max

,num_fold,step,trees,max_depth,threshold,tn,fp,fn,tp,time,MCC,DIS
60,9,60,200,182,38,1808346.0,209742.0,322750.0,1630487.0,2070.046468,0.732701,0.195206
